In [1]:
import pandas as pd
import os
import datetime
from scipy import stats
import numpy as np

In [2]:
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure
def create_flight_time(df, col):    
    liste = []
    for cols in df[col]:
        if pd.isnull(cols):
            liste.append(np.nan)
        elif float(cols) == 2400:
            cols = datetime.time(0,0)
            liste.append(cols)
        else:
            cols = format_heure(cols)
            liste.append(cols)
    return pd.Series(liste)

def concatenate():

    # load csv file containing the airport information
    airport_df = pd.read_csv("./airports.csv")

    #load csv file containing the flight delay information. this will be the main dataframe
    flights_df =  pd.read_csv('/home/chun/Desktop/gpr_testing/flight/flights.csv')
    main_df = flights_df
    # combine these, so that the main dataframe has latitude longitude information
    airport_df = airport_df.set_index("IATA_CODE")
    for loc in ['ORIGIN','DESTINATION']:
        for dir in ['LATITUDE','LONGITUDE']:
            main_df[loc+'_'+dir] =(airport_df[dir][flights_df[loc+"_AIRPORT"][:]]).reset_index()[[dir]]
#     main_df.to_csv("{}.txt".format('processed'))

    main_df['SCHEDULED_DEPARTURE'] = create_flight_time(main_df, 'SCHEDULED_DEPARTURE')
    main_df['SCHEDULED_HOUR']=pd.to_datetime(main_df['SCHEDULED_DEPARTURE'], format='%H:%M:%S').dt.hour
    airline_dummy = pd.get_dummies(main_df['AIRLINE'])
    main_df = main_df.merge(airline_dummy, left_index=True, right_index=True)
#     variables_to_remove = ['YEAR', 'DAY', 'AIRLINE', 'FLIGHT_NUMBER',
#        'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
#        'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'TAXI_OUT',
#        'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 
#        'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
#        'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
#        'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
#        'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
    variables_to_remove = ['YEAR', 'DAY', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
    main_df.drop(variables_to_remove, axis = 1, inplace = True)
    main_df.dropna(inplace = True)

    return main_df

In [3]:
main_df = concatenate()

/home/chun/py/py3env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
main_df2 = pd.get_dummies(main_df, columns=['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'])
main_df2.shape

(5222000, 662)

In [ ]:
main_df2.head(5)

In [ ]:
tmp = main_df2
tmp_y = tmp[['ARRIVAL_DELAY']]
tmp_x = tmp.drop('ARRIVAL_DELAY', axis=1)

t_test_list = []

i = 0
# for column in tmp_x:
#     y = tmp_y
#     xmat = matrix(1, ncol=2, nrow=length(y)) 
#     xmat[,2] = x
#     bhead = solve(t(xmat)%*%xmat, t(xmat)%*%y) 
#     yhead = xmat %*% bhead
#     e1 = y - yhead
#     var1 = sum(e1 * e1) / (length(e1)-2) 
#     sigma2 = solve(t(xmat)%*%xmat) * var1 
#     t1 = bhead[2]/sqrt(sigma2[2,2])
#     return(t1)

In [ ]:
def cnt_t_value(x, y):
#     print(x.shape)
#     print(y.shape)
    xmat = np.vstack((np.ones(len(y)), x)).T
    bhead = np.linalg.solve(np.dot(xmat.T, xmat), np.dot(xmat.T, y)) 
    yhead = np.dot(xmat, bhead)
    e1 = y - yhead
    var1 = np.sum(np.dot(e1.T,e1)) / (len(e1)-2) 
    sigma2 = np.linalg.inv(np.dot(xmat.T, xmat)) * var1 
    t1 = bhead[1,0]/np.sqrt(sigma2[1,1])
    return t1
# cnt_t_value(tmp_x.iloc[:,0], tmp_y)

t_value_list = []
for column in tmp_x:
    print(column)
    x  = np.reshape(tmp_x[[column]].values, len(x))
    t_value = cnt_t_value(x, tmp_y)
    t_value_list.append(t_value)

In [ ]:
aaa = np.array(t_value_list)
np.abs(t_value_list).argsort()[::-1]

high_tvalue_id = np.abs(t_value_list).argsort()[::-1][:35]

In [ ]:
np.array(list(tmp_x))[np.abs(t_value_list).argsort()[::-1][:35]]

In [ ]:
outdf = tmp_x.iloc[:,high_tvalue_id]

In [ ]:
outdf2 = pd.concat([outdf, tmp_y], axis=1)
print(outdf2.shape)
outdf2.head(5)

In [ ]:
outdf2.to_csv("{}.txt".format('processed_ttest'))